In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from time import sleep
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('Email_1.csv')

In [3]:
df

,Price,Item
0,5124.43,"Apple MacBook Pro 16"" - Silber 2019 MVVL2D/A i..."
1,3057.92,"Apple MacBook Pro 16"" - Silber 2019 MVVM2D/A i..."
2,3057.92,"Apple MacBook Pro 16"" - Space Grau 2019 MVVK2D..."
3,2859.00,"Nuevo Apple MacBook Pro (de 16 pulgadas, 16 GB..."
4,2699.00,"Apple - MacBook Pro 15"" (All-in-One Desktop PC..."


In [4]:
X = int(input()) #X will be a variable into a param

0


In [5]:
item = df.loc[X,'Item']
item 

'Apple MacBook Pro 16" - Silber 2019 MVVL2D/A i7 2,6GHz, 16GB RAM, 512GB SSD, Radeon Pro 5300M, macOS - Touch Bar'

In [6]:
price = df.loc[X,'Price']
price

5124.43

In [27]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")

driver = webdriver.Chrome('../chromedriver',options=chrome_options)
driver.get("https://www.amazon.es/")

sleep(3)
driver.find_element_by_xpath("//input[@id=\"twotabsearchtextbox\"]")\
    .send_keys(item)
sleep(2)
driver.find_element_by_xpath("//input[contains(@type,'submit')]")\
    .click()
sleep(2)

driver.find_element_by_class_name("a-dropdown-prompt")\
    .click()
sleep(2)
driver.find_element_by_id("s-result-sort-select_2")\
    .click()
sleep(2)
driver.find_element_by_class_name('a-link-normal.a-text-normal')\
    .click()
sleep(2)

driver.find_element_by_class_name('a-expander-prompt')\
    .click()
sleep(2)



review = driver.find_elements_by_class_name('cr-original-review-content')# reviews to do the nltk


item_description = driver.find_element_by_id('productTitle') # full description of the item

price = driver.find_element_by_id('priceblock_ourprice') # Actual price

label = driver.find_elements_by_class_name('label')

label2 = driver.find_elements_by_class_name('value')
    
url = driver.current_url


In [36]:
reviews = [''.join(r.text.split('\n')) for r in review]


In [41]:
reviews = ''.join(reviews)

In [42]:
reviews

"Perfekter Laptop und Arbeitsmittel für einen stolzen Preis nIch habe mir das Macbook pro 2019 gekauft um mit Ableton einen schnelleren workflow zu haben, davor hatte ich das Macbook pro 2012 also ihr könnt euch vorstellen was das für ein technischer unterschied ist-die Hardware ist sehr wertig und hat sehr viel Stil das Design ist sehr glatt und gut verarbeitet und im gesamten stimmig, die 4 USB slots kann man mit einer USB leisten adapter vervielfältigen was ich mir noch dazu gekauft habe um mein Push 2 benutzen zu können-der Display glänzt in Aktion in allen vorstellbaren Farben-von den Lautsprechern bin ich total überzeugt die sind fast schon so gut wie meine Anlage, überhaupt nicht blechen höhen und tiefen sind klar aber nicht statisch-mit den 16 GB RAM komm ich sehr schnell an meine Ziele und arbeite effizienter es macht extrem spaß ohne ruckeln und Wartezeiten an dem gerät zu arbeitenFazit: im Grunde genommen wirklich ein gerät das den namen high end mit Stolz tragen kann und zu

## Item name

In [9]:
item_description.text


'Apple MacBook Pro 16" - Silber 2019 MVVL2D/A i7 2,6GHz, 16GB RAM, 512GB SSD, Radeon Pro 5300M, macOS - Touch Bar'

## Cleaning price to float type

In [10]:
price = price.text
price
price = price.strip(' €')
price = price.replace('.','')
price = price.replace(',','.')
max_price = float(price)
max_price

5124.43

## Create dataframe from Characteristics

In [11]:
'''
information = []
for l in label2:
    information.append(l.text)
len(information)
'''
information = [label.text for label in label2]


In [12]:
'''
name = []
for l in label:
    name.append(l.text)
len(name)
'''
name = [l.text for l in label]


In [13]:
df = {
    'Characteristics': name,
    'Information':information
}

## Finding the reviews to do nltk sentiment analisis of all reviews found

In [14]:
text = []
a=[]
for r in review:
    text.append(r.text.split('\n'))
for e in text:
    for i in e:
        if type(i)==str:
            a.append(i)
text_german = ''.join(a)


## Google translate import to translate reviews to english

In [15]:
from googletrans import Translator
trans = Translator()
t = trans.translate(text_german)

In [16]:
text=t.text
text

"Perfect Laptop and tools for a premium price nI have bought the Macbook pro in 2019 to have a faster workflow with Ableton, before that I had the Macbook pro in 2012 so you can imagine what kind of a technical difference is-the hardware is very valuable and very has a lot of style, the design is very smooth and well finished and consistent throughout, the 4 USB slots you can afford with a USB adapter reproduce what I myself have not bought this to use for my push 2 to-the display shines in action in every imaginable color from the speakers, I am totally convinced that are almost as good as my conditioning, do not fork out ups and downs but are clearly not static with 16 GB of RAM I come very quickly to my goals and work more efficiently, it makes extremely fun without bucking and waiting times at the device to work conclusion: basically really a device containing the names high end proudly wear can and also any artist or self inspires in his creativity and also a fast workflow allows 

In [17]:
from nltk.stem import SnowballStemmer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")





[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/albertjoselobera/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
sia = SentimentIntensityAnalyzer()


In [19]:
sentiment = sia.polarity_scores(text)
sentiment
sentiment_changed = {'Negativo':sentiment['neg'],'Neutro':sentiment['neu'],'Positivo':sentiment['pos']}

In [20]:
sentiment_changed = {'Positivo':round(sentiment_changed['Positivo']*100/1,2),'Neutro':round(sentiment_changed['Neutro']*100/1,2),'Negativo':round(sentiment_changed['Negativo']*100/1,2)}


In [21]:
df = pd.DataFrame.from_dict(df, orient='index')
df = df.transpose()

In [22]:
product = df.set_index(['Characteristics'])

In [23]:
product.to_csv('Email_2.csv')

In [24]:
import os
import smtplib
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

## Price maximum selected by the user

## Email with product detail

In [ ]:
def send_mail():

    
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')


    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    server.login(sender_email,password)
    
    subject = 'Pythonium, detalles del artículo'
    
    body = f'''Buenos días!\n\nVoy a recordarte el nombre y especificaciones generales del producto escogido: 
    \n{item}\nPrecio actual: {max_price}.\n\n
    Segun mi sistema de sentimiento, las valoraciones tienen un % de:\n
    {sentiment_changed}\n\n\nEnlace al producto:\n {url}
    
    Te adjunto una tabla con los datos técnicos del producto para que le eches un ojo:\n\n\n{product}\n\n 
    Cuando baje al precio deseado te lo recordaré.\n\n Ten un buen día!,\n Pythonium,'''

    
    msg = f'Subject:{subject}\n\n{body}'.encode()
    
    
    server.sendmail(
        sender_email,
        receiver_email,
        msg
    )
    
    print('Hey email has been sent!')
    
    server.quit()

In [ ]:
send_mail()

## Last email

if price goes down

In [ ]:
def send_final_email():

    
    sender_email = os.getenv('emailP')
    receiver_email = os.getenv('email')
    password = os.getenv('PasswordP')


    server = smtplib.SMTP('smtp.gmail.com',587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    
    server.login(sender_email,password)
    
    subject = 'Pythonium, ¡Tienes una oferta!'
    
    body = f'''Buenas noticias!\n\nNuestro producto\n {item}\n\n Se encuentra al siguiente precio {max_price}! un chollazo!, 
si aun sigues interesado te dejo el enlace:\n\n {url}.\n\n\n Ten un buen día!,\n Pythonium,'''

    
    msg = f'Subject:{subject}\n\n{body}'.encode()
    
    
    server.sendmail(
        sender_email,
        receiver_email,
        msg
    )
    
    print('Hey email has been sent!')
    
    server.quit()

In [ ]:
mac = 5000
tu_precio = 4500

def last_email(min_price)
    if (max_price <= min_price):
        send_final_email()
    
    
item
max_price
url  
    
    